In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Mar 23 17:22:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!git clone https://gitlab.com/bigirqu/quranqa.git
!pip install git+https://github.com/neuml/txtai datasets pandas
!pip install farasapy

Cloning into 'quranqa'...
remote: Enumerating objects: 247, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 247 (delta 29), reused 71 (delta 22), pack-reused 163
Receiving objects: 100% (247/247), 241.68 KiB | 1.34 MiB/s, done.
Resolving deltas: 100% (83/83), done.
  Cloning https://github.com/neuml/txtai to /tmp/pip-req-build-3ycfs2mi
  Running command git clone -q https://github.com/neuml/txtai /tmp/pip-req-build-3ycfs2mi
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 325 kB 4.3 MB/s 
     |████████████████████████████████| 596 kB 66.0 MB/s 
     |████████████████████████████████| 3.8 MB 65.9 MB/s 
     |████████████████████████████████| 6.5 MB 54.1 MB/s 
     |████████████████████████████████| 8.6 MB 51.5 MB/s 
     |████████████████████████████████| 67 kB 6.5 MB/s 
     |████████████████████████████

In [5]:
import sys
sys.path.insert(1, '/content/quranqa/code')
import read_write_qrcd
import quranqa22_eval

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from txtai.pipeline import HFTrainer
import torch, string, re, os, json
import operator

from farasa.pos import FarasaPOSTagger 
from farasa.ner import FarasaNamedEntityRecognizer 
from farasa.diacratizer import FarasaDiacritizer 
from farasa.segmenter import FarasaSegmenter 
from farasa.stemmer import FarasaStemmer



In [6]:

train_passage_question_objects = read_write_qrcd.read_JSONL_file('./quranqa/datasets/qrcd_v1.1_train.jsonl')
dev_passage_question_objects = read_write_qrcd.read_JSONL_file('./quranqa/datasets/qrcd_v1.1_dev.jsonl')


Loaded 710 records from ./quranqa/datasets/qrcd_v1.1_train.jsonl
pq_id: 2:8-16_364
pq_id: 2:174-176_364
pq_id: 14:47-52_364
pq_id: 17:12-17_364
pq_id: 18:29-31_364
pq_id: 27:89-93_364
pq_id: 30:38-41_364
pq_id: 39:38-41_364
pq_id: 91:1-10_364
pq_id: 2:8-16_378
pq_id: 2:174-176_378
pq_id: 2:277-281_378
pq_id: 2:284-286_378
pq_id: 5:48-50_378
pq_id: 6:27-32_378
pq_id: 6:100-105_378
pq_id: 6:148-150_378
pq_id: 14:47-52_378
pq_id: 16:35-40_378
pq_id: 17:12-17_378
pq_id: 18:29-31_378
pq_id: 27:89-93_378
pq_id: 30:38-41_378
pq_id: 35:15-18_378
pq_id: 39:38-41_378
pq_id: 41:45-48_378
pq_id: 42:27-31_378
pq_id: 45:12-15_378
pq_id: 47:1-6_378
pq_id: 67:1-5_378
pq_id: 73:15-19_378
pq_id: 74:32-48_378
pq_id: 76:23-31_378
pq_id: 78:38-40_378
pq_id: 81:15-29_378
pq_id: 91:1-10_378
pq_id: 2:17-20_343
pq_id: 10:3-6_343
pq_id: 24:35-38_343
pq_id: 2:25-25_143
pq_id: 55:46-78_143
pq_id: 56:11-26_143
pq_id: 56:27-40_143
pq_id: 2:26-27_231
pq_id: 2:49-52_231
pq_id: 2:53-57_231
pq_id: 2:63-66_231
pq_id: 2:

In [7]:

train_data = [dict({"pq_id": passage_question_object.pq_id,
                    "question": quranqa22_eval.normalize_text(passage_question_object.question),
                    "context": passage_question_object.passage, 
                    "answers": r.text})
              for passage_question_object in train_passage_question_objects
              for r in passage_question_object.answers]


dev_data = [dict({"pq_id": passage_question_object.pq_id,
                  "question": quranqa22_eval.normalize_text(passage_question_object.question),
                  "context": passage_question_object.passage,
                  "answers": r.text})
            for passage_question_object in dev_passage_question_objects
            for r in passage_question_object.answers]



In [8]:
import pandas as pd
dev_data_df = pd.DataFrame(dev_data)
dev_data_df.to_csv('dev.csv', sep='\t')
train_data_df = pd.DataFrame(train_data)
train_data_df.to_csv('train.csv', sep='\t')

!cp *.csv /content/drive/MyDrive

In [9]:
pretrained_models = [#"wissamantoun/araelectra-base-artydiqa", #num_train_epochs=2
                     #"salti/AraElectra-base-finetuned-ARCD", #num_train_epochs=8
                     "gfdgdfgdg/arap_qa_bert_large_v2",
                     #"gfdgdfgdg/arap_qa_bert_v2",
                     #"gfdgdfgdg/arap_qa_bert"
                     ]


def tokenize_function(row,tokenizer):
    return tokenizer.encode_plus(row['question'], row['context'], 
                                 return_tensors='pt', padding=True, 
                                 truncation=True,max_length=512, 
                                 add_special_tokens = True)

topk_n = 20
def get_prediction(row, model, tokenizer):
    inputs = tokenize_function(row,tokenizer)
    output = model(**inputs)
    
    #print(torch.topk(output.start_logits.flatten(), topk_n))
    answer_starts_val = torch.topk(output.start_logits.flatten(), topk_n).values
    answer_ends_val = torch.topk(output.end_logits.flatten(), topk_n).values
    answer_starts = torch.topk(output.start_logits.flatten(), topk_n).indices
    answer_ends = torch.topk(output.end_logits.flatten(), topk_n).indices
    
    answers = []
    rank = 1
    for answer_start,answer_end,answer_start_v,answer_end_v in zip(answer_starts,answer_ends,answer_starts_val,answer_ends_val):
        answer = {}
        ans = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end+1]))
        if len(ans.strip())>0:
            answer["answer"] = ans
            answer["rank"] = rank
            answer["score"] = answer_start_v.item()+answer_end_v.item()
            rank = rank +1
            answers.append(answer)
        if rank == 6:
            break
        #answer = str(answer_start_v.item())+' '+ answer +' '+ str(answer_end_v.item())
    return answers


In [10]:

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    print(tokenizer.decode(prediction[0]))
    print(tokenizer.decode(prediction))
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    exact_match = compute_exact_match(pred, labels)
    f1 = compute_f1(pred, labels)

    return { "f1": f1, "exact_match": exact_match}


In [11]:
from transformers import EarlyStoppingCallback

def Fine_tune(model_name,traindata,devdata,num_train_epochs,batch_size):
    trainer = HFTrainer()

    return trainer(model_name, traindata,devdata,
                   task="question-answering",
                   evaluation_strategy ='steps',
                   eval_steps = 50, # Evaluation and Save happens every 50 steps
                   save_total_limit = 15,
                   learning_rate=2e-5,
                   fp16=True,
                   per_device_train_batch_size=batch_size,
                   per_device_eval_batch_size=batch_size,
                   num_train_epochs=num_train_epochs,
                   weight_decay=0.01,
                   load_best_model_at_end=False, # change this to True after hyperparameter tuning
                   save_strategy='no',
                   compute_metrics=compute_metrics,
                   metric_for_best_model = 'f1',
                   callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
                   #tokenizer=tokenizer
                   )
    #return trainer(model_name, data, task="question-answering", num_train_epochs=num_train_epochs)#num_train_epochs=10

In [12]:
def test_model(tokenizer,model,data):
    result = {}
    for row in data:
        pred = get_prediction(row, model, tokenizer)
        result[row["pq_id"]] = pred
    return result

In [ ]:
model_name = "gfdgdfgdg/arap_qa_bert_large_v2" #num_train_epochs=???
num_train_epochs=7
batch_size=1

print("\n\n###################"+model_name)
tokenizer_salti = AutoTokenizer.from_pretrained(model_name)
model_salti = AutoModelForQuestionAnswering.from_pretrained(model_name)
result = test_model(tokenizer_salti,model_salti,dev_data)
open('./quranqa/datasets/large_v2_RunID.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/large_v2_RunID.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'


model, tokenizer = Fine_tune(model_name,train_data,dev_data,num_train_epochs,batch_size)
os.makedirs('arap_qa_bert_large_v2')
model.save_pretrained('arap_qa_bert_large_v2')
tokenizer.save_pretrained('arap_qa_bert_large_v2')
tokenizer = AutoTokenizer.from_pretrained('./arap_qa_bert_large_v2')
model = AutoModelForQuestionAnswering.from_pretrained('./arap_qa_bert_large_v2')
result = test_model(tokenizer,model,dev_data)
open('./quranqa/datasets/largev2_Run'+str(num_train_epochs)+'F.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/largev2_Run{num_train_epochs}F.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'





###################gfdgdfgdg/arap_qa_bert_large_v2


Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
100% 241M/241M [00:13<00:00, 17.3MiB/s]
[2022-03-21 13:34:02,980 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
Error: Your run file name <large_v2_RunID.json> is incorrect. 
	   Please adopt this naming formt <TeamID_RunID.json> 
	   such that: 
		- TeamID can be an alphanumeric with a length between 3 and 9 characters 
		- RunID  can be an alphanumeric with a length between 2 and 9 characters 
		    For example: bigIR_run01.json
Please review the above warning(s) or error message(s) related to this run file.

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss
1,3.124500,2.722415
2,1.660600,2.841961
3,1.460700,3.140558
4,1.023500,3.693988
5,0.841900,3.978391
6,0.714600,4.797897
7,0.633500,4.849733


[2022-03-21 14:08:58,359 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.5169497488489666, "exact_match": 0.24770642201834864, "f1": 0.4867348265938929}


In [ ]:
model_name = "wissamantoun/araelectra-base-artydiqa" #num_train_epochs=2
num_train_epochs=2
batch_size=1

print("\n\n###################"+model_name)
tokenizer_salti = AutoTokenizer.from_pretrained(model_name)
model_salti = AutoModelForQuestionAnswering.from_pretrained(model_name)
result = test_model(tokenizer_salti,model_salti,dev_data)
open('./quranqa/datasets/wissam_RunID.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/wissam_RunID.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'


model, tokenizer = Fine_tune(model_name,train_data,dev_data,num_train_epochs,batch_size)
os.makedirs('Fine_tuned_wissamantoun')
model.save_pretrained('Fine_tuned_wissamantoun')
tokenizer.save_pretrained('Fine_tuned_wissamantoun')
tokenizer = AutoTokenizer.from_pretrained('./Fine_tuned_wissamantoun')
model = AutoModelForQuestionAnswering.from_pretrained('./Fine_tuned_wissamantoun')
result = test_model(tokenizer,model,dev_data)
open('./quranqa/datasets/wissam_Run'+str(num_train_epochs)+'F.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/wissam_Run{num_train_epochs}F.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'





###################wissamantoun/araelectra-base-artydiqa


Downloading:   0%|          | 0.00/390 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/771 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/743k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/514M [00:00<?, ?B/s]

[2022-03-21 14:10:45,708 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.27996145338482714, "exact_match": 0.027522935779816515, "f1": 0.23780263190542983}


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss
1,2.602400,2.257186
2,1.277500,2.973269


[2022-03-21 14:15:15,989 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.5619226582799326, "exact_match": 0.25688073394495414, "f1": 0.5330240687967356}


In [ ]:
model_name = "salti/AraElectra-base-finetuned-ARCD" #num_train_epochs=11
num_train_epochs=11
batch_size=1

print("\n\n###################"+model_name)
tokenizer_salti = AutoTokenizer.from_pretrained(model_name)
model_salti = AutoModelForQuestionAnswering.from_pretrained(model_name)
result = test_model(tokenizer_salti,model_salti,dev_data)
open('./quranqa/datasets/salti_RunID.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/salti_RunID.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'

model, tokenizer = Fine_tune(model_name,train_data,dev_data,num_train_epochs,batch_size)
os.makedirs('salti')
model.save_pretrained('salti')
tokenizer.save_pretrained('salti')
tokenizer = AutoTokenizer.from_pretrained('./salti')
model = AutoModelForQuestionAnswering.from_pretrained('./salti')
result = test_model(tokenizer,model,dev_data)
open('./quranqa/datasets/salti_Run'+str(num_train_epochs)+'F.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/salti_Run{num_train_epochs}F.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'





###################salti/AraElectra-base-finetuned-ARCD


Downloading:   0%|          | 0.00/390 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/771 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/743k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/514M [00:00<?, ?B/s]

[2022-03-21 14:16:57,805 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.30088128742343634, "exact_match": 0.01834862385321101, "f1": 0.26006557801033875}


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss
1,2.883500,2.446347
2,1.428300,2.628735
3,1.359300,3.676520
4,0.946600,3.887541
5,0.765800,4.347068
6,0.740900,4.473617
7,0.632500,4.489114
8,0.537100,4.692490
9,0.417500,5.135780
10,0.361100,4.958975


[2022-03-21 14:36:01,781 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.6186010092307955, "exact_match": 0.3302752293577982, "f1": 0.5853652091248503}


In [ ]:
'''
model_name = "gfdgdfgdg/arap_qa_bert_v2" #num_train_epochs=9
num_train_epochs=9

print("\n\n###################"+model_name)
tokenizer_salti = AutoTokenizer.from_pretrained(model_name)
model_salti = AutoModelForQuestionAnswering.from_pretrained(model_name)
result = test_model(tokenizer_salti,model_salti,dev_data)
open('./quranqa/datasets/arapv2_RunID.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/arapv2_RunID.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'

model, tokenizer = Fine_tune(model_name,train_data,num_train_epochs)
os.makedirs('arap2')
model.save_pretrained('arap2')
tokenizer.save_pretrained('arap2')
tokenizer = AutoTokenizer.from_pretrained('./arap2')
model = AutoModelForQuestionAnswering.from_pretrained('./arap2')
result = test_model(tokenizer,model,dev_data)
open('./quranqa/datasets/arap2_Run'+str(num_train_epochs)+'F.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/arap2_Run{num_train_epochs}F.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'
'''


'\nmodel_name = "gfdgdfgdg/arap_qa_bert_v2" #num_train_epochs=9\nnum_train_epochs=9\n\nprint("\n\n###################"+model_name)\ntokenizer_salti = AutoTokenizer.from_pretrained(model_name)\nmodel_salti = AutoModelForQuestionAnswering.from_pretrained(model_name)\nresult = test_model(tokenizer_salti,model_salti,dev_data)\nopen(\'./quranqa/datasets/arapv2_RunID.json\' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))\n!python ./quranqa/code/quranqa22_eval.py --run_file \'./quranqa/datasets/arapv2_RunID.json\' --gold_answers_file \'./quranqa/datasets/qrcd_v1.1_dev.jsonl\'\n\nmodel, tokenizer = Fine_tune(model_name,train_data,num_train_epochs)\nos.makedirs(\'arap2\')\nmodel.save_pretrained(\'arap2\')\ntokenizer.save_pretrained(\'arap2\')\ntokenizer = AutoTokenizer.from_pretrained(\'./arap2\')\nmodel = AutoModelForQuestionAnswering.from_pretrained(\'./arap2\')\nresult = test_model(tokenizer,model,dev_data)\nopen(\'./quranqa/datasets/arap2_Run\'+str(num_train_e

In [ ]:
'''
model_name = "gfdgdfgdg/arap_qa_bert" #num_train_epochs=6
num_train_epochs=6

model, tokenizer = Fine_tune(model_name,train_data,num_train_epochs)
os.makedirs('arap')
model.save_pretrained('arap')
tokenizer.save_pretrained('arap')
tokenizer = AutoTokenizer.from_pretrained('./arap')
model = AutoModelForQuestionAnswering.from_pretrained('./arap')
result = test_model(tokenizer,model,dev_data)
open('./quranqa/datasets/arap_Run'+str(num_train_epochs)+'F.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/arap_Run{num_train_epochs}F.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'

'''

'\nmodel_name = "gfdgdfgdg/arap_qa_bert" #num_train_epochs=6\nnum_train_epochs=6\n\nmodel, tokenizer = Fine_tune(model_name,train_data,num_train_epochs)\nos.makedirs(\'arap\')\nmodel.save_pretrained(\'arap\')\ntokenizer.save_pretrained(\'arap\')\ntokenizer = AutoTokenizer.from_pretrained(\'./arap\')\nmodel = AutoModelForQuestionAnswering.from_pretrained(\'./arap\')\nresult = test_model(tokenizer,model,dev_data)\nopen(\'./quranqa/datasets/arap_Run\'+str(num_train_epochs)+\'F.json\' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))\n!python ./quranqa/code/quranqa22_eval.py --run_file \'./quranqa/datasets/arap_Run{num_train_epochs}F.json\' --gold_answers_file \'./quranqa/datasets/qrcd_v1.1_dev.jsonl\'\n\n'

In [17]:
num_train_epochs = 26
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/wissamantoun')
model = AutoModelForQuestionAnswering.from_pretrained('/content/drive/MyDrive/wissamantoun')
result = test_model(tokenizer,model,dev_data)
open('./quranqa/datasets/wissam_Run'+str(num_train_epochs)+'F.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/wissam_Run{num_train_epochs}F.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'



[2022-03-23 17:33:55,342 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.6216961440010439, "exact_match": 0.3486238532110092, "f1": 0.5930611671980399}


In [15]:
num_train_epochs = 22
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/salti')
model = AutoModelForQuestionAnswering.from_pretrained('/content/drive/MyDrive/salti')
result = test_model(tokenizer,model,dev_data)
open('./quranqa/datasets/salti_Run'+str(num_train_epochs)+'F.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/salti_Run{num_train_epochs}F.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'



[2022-03-23 17:29:21,305 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.6300328076934414, "exact_match": 0.3761467889908257, "f1": 0.5964237597614275}


In [16]:
num_train_epochs = 21
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/arap_qa_bert_large_v2')
model = AutoModelForQuestionAnswering.from_pretrained('/content/drive/MyDrive/arap_qa_bert_large_v2')
result = test_model(tokenizer,model,dev_data)
open('./quranqa/datasets/largev2_Run'+str(num_train_epochs)+'F.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/largev2_Run{num_train_epochs}F.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'



[2022-03-23 17:33:03,501 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.5432336817437304, "exact_match": 0.23853211009174313, "f1": 0.5050675496895694}


In [18]:

with open('./quranqa/datasets/wissam_Run26F.json', 'r') as f:
  wissam_RunIDF = json.load(f)

with open('./quranqa/datasets/salti_Run22F.json', 'r') as f:
  salti_RunIDF = json.load(f)

with open('./quranqa/datasets/largev2_Run21F.json', 'r') as f:
  largev2_RunIDF = json.load(f)


  
'''
with open('./quranqa/datasets/arap2_Run9F.json', 'r') as f:
  arap2_RunIDF = json.load(f)

with open('./quranqa/datasets/arap_Run6IDF.json', 'r') as f:
  arap_RunIDF = json.load(f)
'''


"\nwith open('./quranqa/datasets/arap2_Run9F.json', 'r') as f:\n  arap2_RunIDF = json.load(f)\n\nwith open('./quranqa/datasets/arap_Run6IDF.json', 'r') as f:\n  arap_RunIDF = json.load(f)\n"

In [28]:

result = {}
for wissam_pq_dict,_ in zip(wissam_RunIDF,salti_RunIDF):
    ans_score = dict()
    for answer in wissam_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]*0.69
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]*0.69
        
    for answer in salti_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]*0.60
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]*0.60
    '''        
    for answer in arap2_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]
          
    for answer in arap_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]
    '''         
            
    ans_score_sorted = dict(sorted(ans_score.items(), key=operator.itemgetter(1),reverse=True)) 
    answers = []
    rank = 1
    for k in ans_score_sorted:
        answer = {}
        answer["answer"] = k
        answer["rank"] = rank
        answer["score"] = ans_score_sorted[k]
        rank = rank +1
        answers.append(answer)
        if rank == 6:
            break
    result[wissam_pq_dict] = answers
    #break
    
open('./quranqa/datasets/w2s11_RunIDF.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/w2s11_RunIDF.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'



[2022-03-23 17:42:39,941 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.6327294781018409, "exact_match": 0.3577981651376147, "f1": 0.6083613558756685}


In [20]:

result = {}
for wissam_pq_dict,_ in zip(wissam_RunIDF,salti_RunIDF):
    ans_score = dict()
        
    for answer in salti_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]

    for answer in wissam_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]
    '''        
    for answer in arap2_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]
          
    for answer in arap_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]
    '''         
            
    ans_score_sorted = dict(sorted(ans_score.items(), key=operator.itemgetter(1),reverse=True)) 
    answers = []
    rank = 1
    for k in ans_score_sorted:
        answer = {}
        answer["answer"] = k
        answer["rank"] = rank
        answer["score"] = ans_score_sorted[k]
        rank = rank +1
        answers.append(answer)
        if rank == 6:
            break
    result[wissam_pq_dict] = answers
    #break
    
open('./quranqa/datasets/s11w2_RunIDF.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/s11w2_RunIDF.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'



[2022-03-23 17:35:17,960 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.6189742609301768, "exact_match": 0.3577981651376147, "f1": 0.5923817351522657}


In [ ]:

result = {}
for wissam_pq_dict,_,_ in zip(wissam_RunIDF,salti_RunIDF, largev2_RunIDF):
    ans_score = dict()
    for answer in salti_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]  
            
    for answer in wissam_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]
            
    for answer in largev2_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]
    '''
    for answer in arap_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]
    '''         
            
    ans_score_sorted = dict(sorted(ans_score.items(), key=operator.itemgetter(1),reverse=True)) 
    answers = []
    rank = 1
    for k in ans_score_sorted:
        answer = {}
        answer["answer"] = k
        answer["rank"] = rank
        answer["score"] = ans_score_sorted[k]
        rank = rank +1
        answers.append(answer)
        if rank == 6:
            break
    result[wissam_pq_dict] = answers
    #break
    
open('./quranqa/datasets/swl_RunIDF.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/swl_RunIDF.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'



[2022-03-21 14:38:55,072 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.6078078348985986, "exact_match": 0.30275229357798167, "f1": 0.5731745926745947}


In [ ]:

result = {}
for wissam_pq_dict,_,_ in zip(wissam_RunIDF,salti_RunIDF, largev2_RunIDF):
    ans_score = dict()
            
    for answer in largev2_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]

    for answer in salti_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]  
            
    for answer in wissam_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]
    '''
    for answer in arap_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]
    '''         
            
    ans_score_sorted = dict(sorted(ans_score.items(), key=operator.itemgetter(1),reverse=True)) 
    answers = []
    rank = 1
    for k in ans_score_sorted:
        answer = {}
        answer["answer"] = k
        answer["rank"] = rank
        answer["score"] = ans_score_sorted[k]
        rank = rank +1
        answers.append(answer)
        if rank == 6:
            break
    result[wissam_pq_dict] = answers
    #break
    
open('./quranqa/datasets/lsw_RunIDF.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/lsw_RunIDF.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'



[2022-03-21 14:39:27,449 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.6078078348985986, "exact_match": 0.30275229357798167, "f1": 0.5731745926745947}


In [ ]:

result = {}
for wissam_pq_dict,_,_ in zip(wissam_RunIDF,salti_RunIDF, largev2_RunIDF):
    ans_score = dict()
            
    for answer in salti_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]  
            
    for answer in largev2_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]

    for answer in wissam_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]
    '''
    for answer in arap_RunIDF[wissam_pq_dict]:
        if answer["answer"] not in ans_score.keys():
            ans_score[answer["answer"]] = answer["score"]
        elif answer["score"] > ans_score[answer["answer"]] :
            ans_score[answer["answer"]] = answer["score"]
    '''         
            
    ans_score_sorted = dict(sorted(ans_score.items(), key=operator.itemgetter(1),reverse=True)) 
    answers = []
    rank = 1
    for k in ans_score_sorted:
        answer = {}
        answer["answer"] = k
        answer["rank"] = rank
        answer["score"] = ans_score_sorted[k]
        rank = rank +1
        answers.append(answer)
        if rank == 6:
            break
    result[wissam_pq_dict] = answers
    #break
    
open('./quranqa/datasets/wls_RunIDF.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/wls_RunIDF.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'



In [ ]:
!cp /content/quranqa/datasets/base_RunIDF.json content/drive